# Section 506.1 - General
## Building Code
`The floor area of a building shall be determined based on the type of construction, occupancy classification, whether there is an automatic sprinkler system installed throughout the building and the amount of building frontage on public way or open space.`

In [2]:
import rdflib
import pandas as pd
# Load RDF graph
rdf_graph = rdflib.Graph()
rdf_graph.parse("..\..\..\LBD files\smartReview\outputHotel Compliant Revit 2023 Advanced with Spaces.ifc.ttl", format="ttl")

<Graph identifier=N3f2bc1c319314bdd86087ac6de44090b (<class 'rdflib.graph.Graph'>)>

In [4]:
# Getting Construction Type, Occupancy Group, & Sprinkler Type for the building
with open("..\..\..\SPARQL\storeyRule.sparql", "r") as file:
    sparql_query = file.read()

results = rdf_graph.query(sparql_query)

for result in results:
    construction_type = result.constructionType.toPython()
    occupancy_group = result.occupancyGroup.toPython()
    sprinkler_type = result.sprinklerType.toPython() if result.sprinklerType else "none"
    storey_num = result.storeyNumValue.toPython()

    # Splitting the construction type and subtype 
    if "-" in construction_type:
        main_construction_type, subtype = construction_type.rsplit("-", 1)
    else:
        main_construction_type = construction_type
        subtype = ""
if sprinkler_type != "none":
    sprinkler_system = "S" 
else: 
    sprinkler_system = "NS"

In [5]:
building_frontage = 950.58

print(f"This section is considered: Passed")


print(f"\nThe allowable floor area of the building was determined based on:\n" \
                             f"• the type of construction: {construction_type},\n" \
                             f"• the occupancy classification: {occupancy_group},\n" \
                             f"• the availability of a sprinkler system installed throughout the building: {sprinkler_type},\n" \
                             f"• the amount of building frontage on public way or open space: {building_frontage} ft."
                             f"\n\nPlease refer to the subsections for further analysis results.")

This section is considered: Passed

The allowable floor area of the building was determined based on:
• the type of construction: Type V-A,
• the occupancy classification: R-1,
• the availability of a sprinkler system installed throughout the building: 13,
• the amount of building frontage on public way or open space: 950.58 ft.

Please refer to the subsections for further analysis results.


# Section 506.1.2 - Special provisions
## Building Code
`The special provisions of Section 510 permit the use of special conditions that are exempt from, or modify, the specific requirements of this chapter regarding the allowable areas of buildings based on the occupancy classification and type of construction, provided the special condition complies with the provisions specified in Section 510.`

This section is considered: Passed

It is understood that Section 510 permits the use of special conditions to alter the allowable area for the building.


# Section 506.1.3 - Basements
## Building Code
`Basements need not be included in the total allowable floor area of a building provided the total area of such basements does not exceed the area permitted for a one-story above grade plane building.`

This section is considered: Understood

The SMARTreview APR software does not include basement areas as part of the determination of whether or not the total floor area satisfies the allowable area requirement for the building.

In [19]:
# Define the allowable area table
table_506_2 = {
    "A-1": {
        "NS": {
            "Type I": {"A": 15_500, "B": 8_500},
            "Type II": {"A": 14_000, "B": 8_500},
            "Type III": {"A": 15_000, "B": 11_500},
            "Type IV": {"HT": 5_500},
            "Type V": {"A": 5_500}
        },
        "S1": {
            "Type I": {"A": 62_000, "B": 34_000},
            "Type II": {"A": 56_000, "B": 34_000},
            "Type III": {"A": 60_000, "B": 46_000},
            "Type IV": {"HT": 22_000},
            "Type V": {"A": 22_000}
        },
        "SM": {
            "Type I": {"A": 46_500, "B": 25_500},
            "Type II": {"A": 42_000, "B": 25_500},
            "Type III": {"A": 45_000, "B": 34_500},
            "Type IV": {"HT": 16_500},
            "Type V": {"A": 16_500}
        }
    },
    "A-2": {
        "NS": {
            "Type I": {"A": 15_500, "B": 9_500},
            "Type II": {"A": 14_000, "B": 9_500},
            "Type III": {"A": 15_000, "B": 11_500},
            "Type IV": {"HT": 6_000},
            "Type V": {"A": 6_000}
        },
        "S1": {
            "Type I": {"A": 62_000, "B": 38_000},
            "Type II": {"A": 56_000, "B": 38_000},
            "Type III": {"A": 60_000, "B": 46_000},
            "Type IV": {"HT": 24_000},
            "Type V": {"A": 24_000}
        },
        "SM": {
            "Type I": {"A": 46_500, "B": 28_500},
            "Type II": {"A": 42_000, "B": 28_500},
            "Type III": {"A": 45_000, "B": 34_500},
            "Type IV": {"HT": 18_000},
            "Type V": {"A": 18_000}
        }
    },
    "A-3": {
        "NS": {
            "Type I": {"A": 15_500, "B": 9_500},
            "Type II": {"A": 14_000, "B": 9_500},
            "Type III": {"A": 15_000, "B": 11_500},
            "Type IV": {"HT": 6_000},
            "Type V": {"A": 6_000}
        },
        "S1": {
            "Type I": {"A": 62_000, "B": 38_000},
            "Type II": {"A": 56_000, "B": 38_000},
            "Type III": {"A": 60_000, "B": 46_000},
            "Type IV": {"HT": 24_000},
            "Type V": {"A": 24_000}
        },
        "SM": {
            "Type I": {"A": 46_500, "B": 28_500},
            "Type II": {"A": 42_000, "B": 28_500},
            "Type III": {"A": 45_000, "B": 34_500},
            "Type IV": {"HT": 18_000},
            "Type V": {"A": 18_000}
        }
    },
    "A-4": {
        "NS": {
            "Type I": {"A": 15_500, "B": 9_500},
            "Type II": {"A": 14_000, "B": 9_500},
            "Type III": {"A": 15_000, "B": 11_500},
            "Type IV": {"HT": 6_000},
            "Type V": {"A": 6_000}
        },
        "S1": {
            "Type I": {"A": 62_000, "B": 38_000},
            "Type II": {"A": 56_000, "B": 38_000},
            "Type III": {"A": 60_000, "B": 46_000},
            "Type IV": {"HT": 24_000},
            "Type V": {"A": 24_000}
        },
        "SM": {
            "Type I": {"A": 46_500, "B": 28_500},
            "Type II": {"A": 42_000, "B": 28_500},
            "Type III": {"A": 45_000, "B": 34_500},
            "Type IV": {"HT": 18_000},
            "Type V": {"A": 18_000}
        }
    },
    "A-5": {
        "NS": {
            "Type I": {"A": "UL"},
            "Type II": {"A": "UL"},
            "Type III": {"A": "UL"},
            "Type IV": {"HT": "UL"},
            "Type V": {"A": "UL"}
        },
        "S1": {
            "Type I": {"A": "UL"},
            "Type II": {"A": "UL"},
            "Type III": {"A": "UL"},
            "Type IV": {"HT": "UL"},
            "Type V": {"A": "UL"}
        },
        "SM": {
            "Type I": {"A": "UL"},
            "Type II": {"A": "UL"},
            "Type III": {"A": "UL"},
            "Type IV": {"HT": "UL"},
            "Type V": {"A": "UL"}
        }
    },
    "B": {
        "NS": {
            "Type I": {"A": 37_500, "B": 23_000},
            "Type II": {"A": 28_500, "B": 19_000},
            "Type III": {"A": 36_000, "B": 18_000},
            "Type IV": {"HT": 9_000},
            "Type V": {"A": 9_000}
        },
        "S1": {
            "Type I": {"A": 150_000, "B": 92_000},
            "Type II": {"A": 114_000, "B": 76_000},
            "Type III": {"A": 144_000, "B": 72_000},
            "Type IV": {"HT": 36_000},
            "Type V": {"A": 36_000}
        },
        "SM": {
            "Type I": {"A": 112_500, "B": 69_000},
            "Type II": {"A": 85_500, "B": 57_000},
            "Type III": {"A": 108_000, "B": 54_000},
            "Type IV": {"HT": 27_000},
            "Type V": {"A": 27_000}
        }
    },
    "I-1": {
        "NS": {
            "Type I": {"A": 55_000, "B": 19_000},
            "Type II": {"A": 10_000, "B": 16_500},
            "Type III": {"A": 10_000, "B": 18_000},
            "Type IV": {"HT": 10_500},
            "Type V": {"A": 4_500}
        },
        "S1": {
            "Type I": {"A": 220_000, "B": 76_000},
            "Type II": {"A": 40_000, "B": 66_000},
            "Type III": {"A": 40_000, "B": 72_000},
            "Type IV": {"HT": 42_000},
            "Type V": {"A": 18_000}
        },
        "SM": {
            "Type I": {"A": 165_000, "B": 57_000},
            "Type II": {"A": 30_000, "B": 49_500},
            "Type III": {"A": 30_000, "B": 54_000},
            "Type IV": {"HT": 31_500},
            "Type V": {"A": 13_500}
        }
    },
    "I-2": {
        "NS": {
            "Type I": {"A": "UL"},
            "Type II": {"A": 15_000, "B": 11_000},
            "Type III": {"A": 12_000},
            "Type IV": {"A": "NP"},
            "Type V": {"A": 12_000}
        },
        "S1": {
            "Type I": {"A": "UL"},
            "Type II": {"A": 60_000, "B": 44_000},
            "Type III": {"A": 48_000},
            "Type IV": {"A": "NP"},
            "Type V": {"A": 48_000}
        },
        "SM": {
            "Type I": {"A": "UL"},
            "Type II": {"A": 45_000, "B": 33_000},
            "Type III": {"A": 36_000},
            "Type IV": {"A": "NP"},
            "Type V": {"A": 36_000}
        }
    },
    "I-3": {
        "NS": {
            "Type I": {"A": "UL"},
            "Type II": {"A": 15_000, "B": 10_000},
            "Type III": {"A": 10_500, "B": 7_500},
            "Type IV": {"A": 12_000},
            "Type V": {"A": 7_500}
        },
        "S1": {
            "Type I": {"A": 45_000, "B": 40_000},
            "Type II": {"A": 42_000, "B": 30_000},
            "Type III": {"A": 48_000, "B": 30_000},
            "Type IV": {"A": 20_000},
            "Type V": {"A": 15_000}
        },
        "SM": {
            "Type I": {"A": 45_000, "B": 30_000},
            "Type II": {"A": 31_500, "B": 22_500},
            "Type III": {"A": 36_000, "B": 22_500},
            "Type IV": {"A": 15_000},
            "Type V": {"A": 15_000}
        }
    },
    "I-4": {
        "NS": {
            "Type I": {"A": 60_500, "B": 26_500},
            "Type II": {"A": 13_000, "B": 23_500},
            "Type III": {"A": 13_000, "B": 25_500},
            "Type IV": {"A": 18_500},
            "Type V": {"A": 9_000}
        },
        "S1": {
            "Type I": {"A": 121_000, "B": 106_000},
            "Type II": {"A": 52_000, "B": 94_000},
            "Type III": {"A": 52_000, "B": 102_000},
            "Type IV": {"A": 74_000},
            "Type V": {"A": 36_000}
        },
        "SM": {
            "Type I": {"A": 181_500, "B": 79_500},
            "Type II": {"A": 39_000, "B": 70_500},
            "Type III": {"A": 39_000, "B": 76_500},
            "Type IV": {"A": 55_500},
            "Type V": {"A": 27_000}
        }
    },
    "M": {
        "NS": {
            "Type I": {"A": "UL"},
            "Type II": {"A": 21_500, "B": 12_500},
            "Type III": {"A": 18_500},
            "Type IV": {"A": 14_000},
            "Type V": {"A": 9_000}
        },
        "S1": {
            "Type I": {"A": "UL"},
            "Type II": {"A": 86_000, "B": 50_000},
            "Type III": {"A": 74_000},
            "Type IV": {"A": 56_000},
            "Type V": {"A": 36_000}
        },
        "SM": {
            "Type I": {"A": "UL"},
            "Type II": {"A": 64_500, "B": 37_500},
            "Type III": {"A": 55_500},
            "Type IV": {"A": 42_000},
            "Type V": {"A": 27_000}
        }
    },
    "R-1": {
        "NS": {
            "Type I": {"A": 24_000, "B": 16_000},
            "Type II": {"A": 24_000, "B": 16_000},
            "Type III": {"A": 20_500, "B": 12_000},
            "Type IV": {"A": 7_000},
            "Type V": {"A": 7_000}
        },
        "S1": {
            "Type I": {"A": 96_000, "B": 64_000},
            "Type II": {"A": 96_000, "B": 64_000},
            "Type III": {"A": 82_000, "B": 48_000},
            "Type IV": {"A": 28_000},
            "Type V": {"A": 28_000}
        },
        "SM": {
            "Type I": {"A": 72_000, "B": 48_000},
            "Type II": {"A": 72_000, "B": 48_000},
            "Type III": {"A": 61_500, "B": 36_000},
            "Type IV": {"A": 21_000},
            "Type V": {"A": 21_000}
        }
    },
    "R-2": {
        "NS": {
            "Type I": {"A": 24_000, "B": 16_000},
            "Type II": {"A": 24_000, "B": 16_000},
            "Type III": {"A": 20_500, "B": 12_000},
            "Type IV": {"A": 7_000},
            "Type V": {"A": 7_000}
        },
        "S1": {
            "Type I": {"A": 96_000, "B": 64_000},
            "Type II": {"A": 96_000, "B": 64_000},
            "Type III": {"A": 82_000, "B": 48_000},
            "Type IV": {"A": 28_000},
            "Type V": {"A": 28_000}
        },
        "SM": {
            "Type I": {"A": 72_000, "B": 48_000},
            "Type II": {"A": 72_000, "B": 48_000},
            "Type III": {"A": 61_500, "B": 36_000},
            "Type IV": {"A": 21_000},
            "Type V": {"A": 21_000}
        }
    },
    "R-3": {
        "NS": {
            "Type I": {"A": "UL"},
            "Type II": {"A": "UL"},
            "Type III": {"A": "UL"},
            "Type IV": {"A": "UL"},
            "Type V": {"A": "UL"}
        },
        "S1": {
            "Type I": {"A": "UL"},
            "Type II": {"A": "UL"},
            "Type III": {"A": "UL"},
            "Type IV": {"A": "UL"},
            "Type V": {"A": "UL"}
        },
        "SM": {
            "Type I": {"A": "UL"},
            "Type II": {"A": "UL"},
            "Type III": {"A": "UL"},
            "Type IV": {"A": "UL"},
            "Type V": {"A": "UL"}
        }
    },
    "R-4": {
        "NS": {
            "Type I": {"A": 24_000, "B": 16_000},
            "Type II": {"A": 24_000, "B": 16_000},
            "Type III": {"A": 20_500, "B": 12_000},
            "Type IV": {"A": 7_000},
            "Type V": {"A": 7_000}
        },
        "S1": {
            "Type I": {"A": 96_000, "B": 64_000},
            "Type II": {"A": 96_000, "B": 64_000},
            "Type III": {"A": 82_000, "B": 48_000},
            "Type IV": {"A": 28_000},
            "Type V": {"A": 28_000}
        },
        "SM": {
            "Type I": {"A": 72_000, "B": 48_000},
            "Type II": {"A": 72_000, "B": 48_000},
            "Type III": {"A": 61_500, "B": 36_000},
            "Type IV": {"A": 21_000},
            "Type V": {"A": 21_000}
        }
    },
    "S-1": {
        "NS": {
            "Type I": {"A": 48_000, "B": 26_000},
            "Type II": {"A": 17_500, "B": 26_000},
            "Type III": {"A": 17_500, "B": 25_500},
            "Type IV": {"A": 14_000},
            "Type V": {"A": 9_000}
        },
        "S1": {
            "Type I": {"A": 192_000, "B": 104_000},
            "Type II": {"A": 70_000, "B": 104_000},
            "Type III": {"A": 70_000, "B": 102_000},
            "Type IV": {"A": 56_000},
            "Type V": {"A": 36_000}
        },
        "SM": {
            "Type I": {"A": 144_000, "B": 78_000},
            "Type II": {"A": 52_500, "B": 78_000},
            "Type III": {"A": 52_500, "B": 76_500},
            "Type IV": {"A": 42_000},
            "Type V": {"A": 27_000}
        }
    },
    "S-2": {
        "NS": {
            "Type I": {"A": 79_000, "B": 39_000},
            "Type II": {"A": 26_000, "B": 39_000},
            "Type III": {"A": 26_000, "B": 38_500},
            "Type IV": {"A": 21_000},
            "Type V": {"A": 13_500}
        },
        "S1": {
            "Type I": {"A": 316_000, "B": 156_000},
            "Type II": {"A": 104_000, "B": 156_000},
            "Type III": {"A": 104_000, "B": 154_000},
            "Type IV": {"A": 84_000},
            "Type V": {"A": 54_000}
        },
        "SM": {
            "Type I": {"A": 237_000, "B": 117_000},
            "Type II": {"A": 78_000, "B": 117_000},
            "Type III": {"A": 78_000, "B": 115_500},
            "Type IV": {"A": 63_000},
            "Type V": {"A": 40_500}
        }
    },
    "U": {
        "NS": {
            "Type I": {"A": 35_500, "B": 19_000},
            "Type II": {"A": 8_500, "B": 14_000},
            "Type III": {"A": 8_500, "B": 18_000},
            "Type IV": {"A": 9_000},
            "Type V": {"A": 5_500}
        },
        "S1": {
            "Type I": {"A": 142_000, "B": 76_000},
            "Type II": {"A": 34_000, "B": 56_000},
            "Type III": {"A": 34_000, "B": 72_000},
            "Type IV": {"A": 36_000},
            "Type V": {"A": 22_000}
        },
        "SM": {
            "Type I": {"A": 106_500, "B": 57_000},
            "Type II": {"A": 25_500, "B": 42_000},
            "Type III": {"A": 25_500, "B": 54_000},
            "Type IV": {"A": 27_000},
            "Type V": {"A": 16_500}
        }
    }
}


# Section 506.2 - Allowable area determination
## Building Code
The allowable area of a building shall be determined in accordance with the applicable provisions of Sections 506.2.1 through 506.2.4 and Section 506.3. 

### TABLE 506.2a, b ALLOWABLE AREA FACTOR (At = NS, S1, S13R, or SM, as applicable) IN SQUARE FEET 

In [21]:
# Table 506.2 Display
from IPython.display import display, HTML

rows = []

for occupancy_group, systems in table_506_2.items():
    for sprinkler_system, types in systems.items():
        for construction_type, subtypes in types.items():
            for subtype, area_limit in subtypes.items():
                rows.append({
                    "Occupancy Group": occupancy_group,
                    "Sprinkler System": sprinkler_system,
                    "Construction Type": construction_type,
                    "Subtype": subtype,
                    "Area Limit": area_limit
                })

df = pd.DataFrame(rows)

# Display the DataFrame as a scrollable table
def render_scrollable_table(dataframe, max_height=300):
    display(HTML(f"""
    <div style="max-height: {max_height}px; overflow-y: scroll;">
        {dataframe.to_html(index=False)}
    </div>
    """))

render_scrollable_table(df)

Occupancy Group,Sprinkler System,Construction Type,Subtype,Area Limit
A-1,NS,Type I,A,15500
A-1,NS,Type I,B,8500
A-1,NS,Type II,A,14000
A-1,NS,Type II,B,8500
A-1,NS,Type III,A,15000
A-1,NS,Type III,B,11500
A-1,NS,Type IV,HT,5500
A-1,NS,Type V,A,5500
A-1,S1,Type I,A,62000
A-1,S1,Type I,B,34000


**Note:** UL = Unlimited; NP = Not permitted;  
For SI: 1 square foot = 0.0929 m².  

- **NS** = Buildings not equipped throughout with an automatic sprinkler system  
- **S1** = Buildings a maximum of one story above grade plane equipped throughout with an automatic sprinkler system installed in accordance with Section 903.3.1.1  
- **SM** = Buildings two or more stories above grade plane equipped throughout with an automatic sprinkler system installed in accordance with Section 903.3.1.1  
- **S13R** = Buildings equipped throughout with an automatic sprinkler system installed in accordance with Section 903.3.1.2  

**Footnotes:**

a. See Chapters 4 and 5 for specific exceptions to the allowable height in this chapter.  
b. See Section 903.2 for the minimum thresholds for protection by an automatic sprinkler system for specific occupancies.  
c. New Group H occupancies are required to be protected by an automatic sprinkler system in accordance with Section 903.2.5.  
d. The NS value is only for use in evaluation of existing building area in accordance with the International Existing Building Code.  
e. New Group I-1 and I-3 occupancies are required to be protected by an automatic sprinkler system in accordance with Section 903.2.6. For new Group I-1 occupancies, Condition 1, see Exception 1 of Section 903.2.6.  
f. New and existing Group I-2 occupancies are required to be protected by an automatic sprinkler system in accordance with Section 903.2.6 and Section 1103.5 of the International Fire Code.  
g. New Group I-4 occupancies see Exceptions 2 and 3 of Section 903.2.6.  
h. New Group R occupancies are required to be protected by an automatic sprinkler system in accordance with Section 903.2.8.

In [32]:
with open("..\..\SPARQL\\uniqueOccupancy.sparql", "r") as file:
    sparql_query = file.read()

print(f"This section is considered: Passed"
f"\n\nThe allowable area of the building was determined in accordance with Section 506.2.4 as there is more than one occupancy in the multi-story building: \n")
results = rdf_graph.query(sparql_query)

# Process and format the results
story_dict = {}

for row in results:
    level_value = row[0]
    occupancies = row[1].split(", ")
    
    if level_value not in story_dict:
        story_dict[level_value] = []
    
    story_dict[level_value].extend(occupancies)

# Print the results in the desired format
for level, occupancies in sorted(story_dict.items()):
    unique_occupancies = sorted(set(occupancies))  # Remove duplicates and sort for better display
    num_unique_occupancies = len(unique_occupancies)  # Count the number of unique occupancies
    print(f"•Story: {level} ")
    for occupancy in unique_occupancies:
        print(f"◾Occupancy: {occupancy}")
    print()

This section is considered: Passed

The allowable area of the building was determined in accordance with Section 506.2.4 as there is more than one occupancy in the multi-story building: 

•Story: 01 
◾Occupancy: A-2
◾Occupancy: A-3
◾Occupancy: B
◾Occupancy: R-1

•Story: 02 
◾Occupancy: R-1

•Story: 03 
◾Occupancy: R-1

•Story: 04 
◾Occupancy: R-1



# Section 506.2.1 - Single-occupancy, one-story buildings
## Building Code
```
The allowable area of a single-occupancy building with no more than one story above grade plane shall be determined in accordance with Equation 5-1: 

Aa = At + (NS x If)  (Equation 5-1)

where: 

Aa = Allowable area (square feet). 
At = Tabular allowable area factor (NS, S1, orS13R value, as applicable) in accordance with Table 506.2. 
NS = Tabular allowable area factor in accordancewith Table 506.2 for nonsprinklered building (regardless of whether thebuilding is sprinklered). 
If = Area factor increase due to frontage(percent) as calculated in accordance with Section 506.3. 
```

In [ ]:
if storey_num.lstrip('0') == 1: 
    if num_unique_occupancies == 1: 
        result = "Applicable"
    else:
        result = "Not Applicable"
else: 
    result = "Not Applicable"
    


In [5]:
import rdflib
import pandas as pd
# Load RDF graph
rdf_graph = rdflib.Graph()
rdf_graph.parse("..\..\LBD files\smartReview\outputHotel Compliant Revit 2023 Advanced with Spaces.ifc.ttl", format="ttl")

<Graph identifier=Nd19e33c4365d4a5aa7e18d4b8ae1d402 (<class 'rdflib.graph.Graph'>)>

In [10]:
with open("..\..\SPARQL\\spaceAreaSum.sparql", "r") as file:
    sparql_query = file.read()

results = rdf_graph.query(sparql_query)

# Prepare data for the table
data = {
    "Story": [],
    "Sum": []
}

# Process SPARQL results
for row in results:
    story = str(row["levelValue"]).lstrip('0')  # Remove leading zeros
    area = row["totalArea"]

    data["Story"].append(story)
    data["Sum"].append(area)

df = pd.DataFrame(data)
df

,Story,Sum
0,1,45748.075662204650761
1,3,29796.854063573138566
2,2,29794.857535795623496
3,4,29796.854063573138536
